In [ ]:
# import sys
# sys.path.insert(0, '/Users/niloufar/Desktop/mlops/mlops')

In [ ]:
pip install tfx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 4.2 MB/s 
     |████████████████████████████████| 1.4 MB 40.4 MB/s 
     |████████████████████████████████| 1.5 MB 46.1 MB/s 
     |████████████████████████████████| 21.6 MB 1.2 MB/s 
     |████████████████████████████████| 439 kB 54.0 MB/s 
     |████████████████████████████████| 206 kB 68.4 MB/s 
     |████████████████████████████████| 636 kB 57.8 MB/s 
     |████████████████████████████████| 2.3 MB 54.3 MB/s 
     |████████████████████████████████| 135 kB 65.9 MB/s 
     |████████████████████████████████| 135 kB 68.2 MB/s 
     |████████████████████████████████| 60 kB 6.4 MB/s 
     |████████████████████████████████| 1.7 MB 45.6 MB/s 
     |████████████████████████████████| 6.6 MB 30.1 MB/s 
     |████████████████████████████████| 147 kB 62.3 MB/s 
     |████████████████████████████████| 11.0 MB 47.0 MB/s 
     |███████████████████████████

In [ ]:
import tensorflow as tf
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import pandas as pd

import os
import pprint

In [ ]:
# import inspect
# inspect.getfile(tensorflow_model_analysis)

In [ ]:
# pip install ml-metadata
# pip install apache-beam

In [ ]:
import inspect
import ml_metadata
inspect.getfile(ml_metadata)

'/usr/local/lib/python3.7/dist-packages/ml_metadata/__init__.py'

In [ ]:
pp = pprint.PrettyPrinter()

In [ ]:
_pipeline_root = './pipeline/'
_data_root = '/content/drive/MyDrive/mlops/course2/week2/data/'
_data_filepath = os.path.join(_data_root, 'adult.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(_data_filepath, skipinitialspace=True)

In [ ]:
headerList = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label']

In [ ]:
df = pd.read_csv(_data_root +"adult.csv", skipinitialspace=True)

In [ ]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [ ]:
df.loc[-0.5] = [39, 
                'State-gov', 
                 77516, 
                'Bachelors', 
                 13, 
                'Never-married', 
                'Adm-clerical', 
                'Not-in-family', 
                'White', 
                'Male', 
                 2174, 
                 0, 
                 40, 
                'United-States', 
                 '<=50K']
df = df.sort_index().reset_index(drop=True)

In [ ]:
df.to_csv( _data_root +"adult.csv", header=headerList, index=False)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
!head {_data_filepath}

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husban